Notebook to scrape emails from given gmail

In [14]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
import base64
import os.path
import pickle
import time

In [15]:

SCOPES = ['https://www.googleapis.com/auth/gmail.readonly']

In [16]:
def get_gmail_service():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                '../credentials.json', SCOPES)
            creds = flow.run_local_server(port=8080)
        
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    return build('gmail', 'v1', credentials=creds)

In [17]:
def process_new_emails(service, last_check_time):
    query = f'after:{int(last_check_time)}'
    results = service.users().messages().list(userId='me', q=query).execute()
    messages = results.get('messages', [])
    
    for message in messages:
        msg = service.users().messages().get(userId='me', id=message['id']).execute()
        if 'payload' in msg:
            body = get_email_body(msg['payload'])
            if body:
                # Process the email body here
                print(f"New email received: {body}")

In [18]:
def get_email_body(payload):
    if 'parts' in payload:
        for part in payload['parts']:
            if part['mimeType'] == 'text/plain':
                return base64.urlsafe_b64decode(part['body']['data']).decode()
    elif payload['mimeType'] == 'text/plain':
        return base64.urlsafe_b64decode(payload['body']['data']).decode()
    return None

In [ ]:
def monitor_inbox():
    service = get_gmail_service()
    last_check_time = time.time()
    
    while True:
        process_new_emails(service, last_check_time)
        last_check_time = time.time()
        time.sleep(60)  # Check every minute

In [20]:
# code to load gmail once
service = get_gmail_service()
last_check_time = time.time()
process_new_emails(service, last_check_time)




Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=501287711383-3e9stqph39an2d4ihd5ett1ntpe084q1.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.readonly&state=zxYj2ljCGAu1Qw5gtFLOVIMmUzYVOB&access_type=offline


KeyboardInterrupt: 

In [1]:










if __name__ == '__main__':
    monitor_inbox()